In [1]:
# Dependences
import os
import json
import requests as req
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
from time import time

# Prepare SQL connection string to be used on the functions
CONN_STRING_PATH = '../config/sentinel-conn-string.txt'

with open(CONN_STRING_PATH, 'r') as fid:
    conn_string = fid.read()
    
connection = create_engine(conn_string, pool_recycle=3600).connect()

In [15]:
QUERY = """
        SELECT 
             to_timestamp(b.timestamp) AS timestamp
            ,ce.circulating_fil::NUMERIC / 1e18 AS circulating_fil
            ,ce.vested_fil::NUMERIC / 1e18 AS vested_fil
            ,ce.mined_fil::NUMERIC / 1e18 AS mined_fil
            ,ce.burnt_fil::NUMERIC / 1e18 AS burnt_fil
            ,ce.locked_fil::NUMERIC / 1e18 AS locked_fil
              
        FROM chain_economics ce
        LEFT JOIN block_headers b 
            ON b.parent_state_root = ce.parent_state_root 
            """

df = ( pd.read_sql(QUERY, connection) )
df.head()

,timestamp,circulating_fil,vested_fil,mined_fil,burnt_fil,locked_fil,total_power
0,2020-11-16 21:25:30+00:00,3.298818e+07,2.537157e+07,1.177528e+07,4.027086e+06,1.719821e+07,NaN
1,2020-11-16 21:25:30+00:00,3.298818e+07,2.537157e+07,1.177528e+07,4.027086e+06,1.719821e+07,NaN
2,2020-11-16 21:25:30+00:00,3.298818e+07,2.537157e+07,1.177528e+07,4.027086e+06,1.719821e+07,NaN
3,2020-11-16 21:25:30+00:00,3.298818e+07,2.537157e+07,1.177528e+07,4.027086e+06,1.719821e+07,NaN
4,2020-11-16 21:25:30+00:00,3.298818e+07,2.537157e+07,1.177528e+07,4.027086e+06,1.719821e+07,NaN


In [4]:
df.shape

(977157, 6)

In [20]:
query = """
    SELECT 
         MIN(bh.timestamp) AS time
        ,AVG(total_qa_bytes_power::numeric) * 2^(-50) AS total_power
        ,AVG(total_qa_bytes_committed::numeric) * 2^(-50) as total_committed
        ,AVG(qa_smoothed_position_estimate::numeric) * 2^(-128) * 2^(-50) AS position_estimate
        ,AVG(total_qa_bytes_committed::numeric / total_qa_bytes_power::numeric) as total_committed_over_power
        ,AVG(qa_smoothed_position_estimate::numeric * 2^(-128) / total_qa_bytes_power::numeric) AS position_estimate
        ,AVG(cp.qa_smoothed_velocity_estimate::numeric * 2^(-128) * 2^(-50)) AS velocity_estimate
        

    FROM chain_powers cp
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = cp.state_root
        
    GROUP BY date_trunc('hour',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,time,total_power,total_committed,position_estimate,total_committed_over_power,position_estimate,velocity_estimate
0,1.598306e+09,0.703125,0.703125,0.941073,1.000000,1.338415,0.003659
1,1.598310e+09,0.629112,0.629112,1.343444,1.000000,2.238436,0.003643
2,1.598314e+09,0.468750,0.468750,1.648040,1.000000,3.515818,0.003612
3,1.598317e+09,0.468750,0.468757,1.933061,1.000014,4.123863,0.003567
4,1.598321e+09,0.468750,0.469831,2.181303,1.002306,4.653446,0.003512


In [21]:
df.shape

(2529, 7)

In [2]:
query = """
    SELECT 
         COUNT(*)       

    FROM chain_powers cp
    
"""
df = ( pd.read_sql(query, connection) )
df.head()

,count
0,208603


In [24]:
query = """
    SELECT
         MIN( bh.timestamp ) AS time
        ,AVG( cr.new_reward::numeric * 1e-18 ) as Per_Epoch_Reward_Actual
        ,AVG( cr.new_reward_smoothed_position_estimate::numeric * 2^(-128) * 1e-18 ) as Per_Epoch_Reward_Position_Estimate
              
        FROM chain_rewards cr
        LEFT JOIN block_headers bh
            ON bh.parent_state_root = cr.state_root
        
        GROUP BY date_trunc('hour', to_timestamp(bh.timestamp))
        
        ORDER BY time ASC
"""
df = ( pd.read_sql(query, connection) )
df.head()

,time,per_epoch_reward_actual,per_epoch_reward_position_estimate
0,1.598306e+09,36.271819,36.266576
1,1.598310e+09,36.270711,36.267119
2,1.598314e+09,36.268941,36.267388
3,1.598317e+09,36.268461,36.267554
4,1.598321e+09,36.268070,36.267657


In [23]:
df.shape

(2529, 2)

In [3]:
query = """
    SELECT 
         COUNT(*)       

    FROM chain_rewards cr
    
"""
df = ( pd.read_sql(query, connection) )
df.head()

,count
0,237979


In [33]:
query = """
    SELECT 
         date_trunc( 'hour',  to_timestamp( bh.timestamp ) ) AS time_stamp
         ,COUNT( info.expiration_epoch )
        
        
    FROM miner_sector_infos as info
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = info.state_root
    
    GROUP BY date_trunc('hour',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,time_stamp,count
0,NaT,1008
1,2020-10-21 02:00:00+00:00,66618
2,2020-09-26 09:00:00+00:00,92228
3,2020-09-29 10:00:00+00:00,121136
4,2020-09-22 14:00:00+00:00,94492


In [34]:
df.shape

(1070, 2)

In [4]:
query = """
    SELECT 
         date_trunc( 'minute',  to_timestamp( bh.timestamp ) ) AS date
        ,COUNT( CASE WHEN info.last_update_epoch > 0 THEN deal_id ELSE NULL END ) AS number_of_deals_made
        ,COUNT( CASE WHEN info.slash_epoch > 0 THEN deal_id else NULL END ) AS number_of_terminated_deals
    
    FROM market_deal_states as info
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = info.state_root
    
    GROUP BY date_trunc('minute',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,date,number_of_deals_made,number_of_terminated_deals
0,2020-08-24 22:00:00+00:00,0,0
1,2020-08-24 22:55:00+00:00,4608,0
2,2020-08-25 06:17:00+00:00,0,0
3,2020-08-25 06:25:00+00:00,0,0
4,2020-08-25 07:23:00+00:00,0,0


In [5]:
df.shape

(104222, 3)

In [43]:
query = """
    SELECT 
         COUNT(*)
        
    FROM market_deal_states as info
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = info.state_root
    
    WHERE
        info.last_update_epoch > 0
"""
df = ( pd.read_sql(query, connection) )
df.head()

,count
0,149981352


In [7]:
query = """
    SELECT
         date_trunc('minute',  to_timestamp(bh.timestamp))
        ,COUNT(info.expiration_epoch) AS Upcoming_Sector_Expiration
       
    FROM miner_sector_infos as info
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = info.state_root
        
    GROUP BY date_trunc('minute',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,date_trunc,upcoming_sector_expiration
0,NaT,1008
1,2020-10-08 22:11:00+00:00,3072
2,2020-10-11 01:19:00+00:00,13178
3,2020-10-11 14:56:00+00:00,1776
4,2020-10-19 15:38:00+00:00,2772


In [8]:
df.shape

(63926, 2)

In [9]:
query = """
    SELECT
         date_trunc('minute',  to_timestamp(bh.timestamp)) time_stamp
        ,COUNT(mdp.is_verified) filter (where mdp.is_verified::BOOLEAN) / COUNT(mdp.deal_id) AS verified_fraction
        
    FROM market_deal_proposals as mdp
    
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = mdp.state_root
        
    GROUP BY date_trunc('minute',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,time_stamp,verified_fraction
0,NaT,0
1,2020-10-08 22:11:00+00:00,0
2,2020-09-08 15:55:00+00:00,0
3,2020-10-11 01:19:00+00:00,0
4,2020-10-11 14:56:00+00:00,0


In [10]:
df.shape

(82116, 2)

In [13]:
query = """
    SELECT 
         COUNT(*)       

    FROM messages
    
"""
df = ( pd.read_sql(query, connection) )
df.head()

,count
0,141056286
